# Stock fetching, Cleaning, and Storage
#### This notebook is going to:

###### Fetch 3 non consecutive 2 month slices of Stock data for 2 companies from the year 2024, this example will use APPLE and Amazon
###### Each of these months will be proccessed differently
###### The first month will be written directly to a SQL table
###### The second month will be written to a JSON file
###### The third month will be written to a CSV file

###### We will then load and randomly scramble each of the APPLE files before appending them the SQL table

###### Finally we are going to build a dataframe from the SQL table, determine the missing months, make the appropriate API calls to retrieve the missing months, complete the data frame and update the SQL table

### Imports & Environment Variables

In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd

sys.path.append(os.path.abspath('..'))
# Import handlers
from scripts.handlers.stockHandler import AVStockDataHandler
from scripts.handlers.storageHandler import storageHandler
from scripts.handlers.helperHandler import helperHandler
from scripts.handlers.scrambleHandler import scrambleHandler
from scripts.handlers.SQLHandler import SQLHandler
from scripts.handlers.cleaningHandler import cleaningHandler

# Load environment variables
load_dotenv('../config.env')

# Retrieve the API key and DB_PATH from the environment variables
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
DB_PATH = os.getenv("DB_PATH")
project_root = Path(sys.path[-1])
DB_PATH = project_root / DB_PATH

# Raise an error if the API key or DB_PATH is not found
if not API_KEY:
    raise ValueError("API key not found. Please set the ALPHA_VANTAGE_API_KEY in config.env.")

if not DB_PATH:
    raise ValueError("Database path not found. Please set the DB_PATH in config.env.")

### Handlers, Tickers, And Date Ranges

In [2]:
# Initialize the handlers
stock_handler = AVStockDataHandler(API_KEY)
storage_handler = storageHandler()
SQL_handler = SQLHandler(DB_PATH)
scramble_handler = scrambleHandler()
helper_handler = helperHandler()
cleaning_handler = cleaningHandler()

# Tickers to fetch data for
tickers = ['AAPL', 'AMZN']

# Date range for fetching data
date_ranges = [
    ('2024-01-01', '2024-02-29'),
    ('2024-05-01', '2024-06-30'),
    ('2024-09-01', '2024-10-31')
]

Database created at: /home/eowre/Documents/DataEngineering/AVStockTracker/SQL_data/stock_data.db


### Fetching and Storing ticker data

In [3]:

# Fetch the data, 
sliced_ticker_data = stock_handler.fetch_multiple_tickers(tickers, date_ranges)

# Using heloperHandler to slice the data and create sets
sets = helper_handler.create_sets(sliced_ticker_data, date_ranges)

SQL_handler.save_dfs_to_table(sets[0])
storage_handler.multiple_dfs_to_csv_and_json(sets[1], file_type='csv')
storage_handler.multiple_dfs_to_csv_and_json(sets[2], file_type='json')

Fetching full data for AAPL...
Fetching full data for AMZN...
/home/eowre/Documents/DataEngineering/AVStockTracker/SQL_data/stock_data.db
DataFrame saved to table: AAPL
/home/eowre/Documents/DataEngineering/AVStockTracker/SQL_data/stock_data.db
DataFrame saved to table: AMZN
No new metadata to add.
No new metadata to add.


### File Retrieval

In [4]:
# Define regex patterns for locating AAPL csv and json files
AAPL_pattern = r".*AAPL.*\.(csv|json)$" # Matches any CSV file starting with "AAPL"

# Use the helper handler to locate files matching the patterns
aapl_files = helper_handler.find_files(str(project_root / "raw_data"), AAPL_pattern)
print(f"Found {len(aapl_files)} files matching the pattern '{AAPL_pattern}' in 'raw_data' directory.")
# Build dataframes from the located files
aapl_dfs = []
for file in aapl_files:
    df = storage_handler.df_builder(file)
    df.reset_index(inplace=True)
    aapl_dfs.append(df)
# Concatenate the dataframes into a single dataframe
aapl_df = pd.concat(aapl_dfs)

Found 2 files matching the pattern '.*AAPL.*\.(csv|json)$' in 'raw_data' directory.


### Scrambling data

###### NOTE: This scramble method can return a new object but the operations are performed in place on the df

In [5]:
# Print the concatenated dataframe
print(aapl_df)

# Scramble the data
scramble_handler.scramble_df(aapl_df)

# Print the scrambled dataframe
print(aapl_df)


         date     open     high     low   close       volume
0  2024-05-01  169.580  172.705  169.11  169.30   50383147.0
1  2024-05-02  172.510  173.415  170.89  173.03   94214915.0
2  2024-05-03  186.645  187.000  182.66  183.38  163224109.0
3  2024-05-06  182.354  184.200  180.42  181.71   78569667.0
4  2024-05-07  183.450  184.900  181.32  182.40   77305771.0
..        ...      ...      ...     ...     ...          ...
38 2024-10-25  229.740  233.220  229.57  231.41   38802304.0
39 2024-10-28  233.320  234.730  232.55  233.40   36087134.0
40 2024-10-29  233.100  234.325  232.32  233.67   35417247.0
41 2024-10-30  232.610  233.470  229.55  230.10   47070907.0
42 2024-10-31  229.340  229.830  225.37  225.91   64370086.0

[84 rows x 6 columns]
                date        open        high         low       close  \
0         2024-05-01      169.58     172.705  169.11 USD       169.3   
1         2024-05-02  USD 172.51  USD 173.41     170.89$      173.03   
2         05/03/2024     $186

### Cleaning the scrambled data And saving to SQL table

###### Simple methdo to clean the data by using a set list of common date and currency formats

In [6]:
cleaning_handler.clean_data(aapl_df)

# Set index to 'date' and sort the dataframe
aapl_df.set_index('date', inplace=True)
aapl_df.sort_index(inplace=True)

# Print the cleaned dataframe
print(aapl_df)

SQL_handler.save_df_to_table(aapl_df, tickers[0])

               open     high      low    close       volume
date                                                       
2024-05-01  169.580  172.705  169.110  169.300   50383147.0
2024-05-02  172.510  173.410  170.890  173.030   94214915.0
2024-05-03  186.650  187.000  182.660  183.380  163224109.0
2024-05-06  182.354  184.200  180.420  181.710   78569667.0
2024-05-07  183.450  184.900  181.320  182.400   77305771.0
...             ...      ...      ...      ...          ...
2024-10-25  211.500  214.860  210.640  213.250   38802304.0
2024-10-28  214.690  215.739  212.350  214.100   36087134.0
2024-10-29  215.770  216.070  210.300  210.620   35417247.0
2024-10-30  232.610  233.470  229.550  230.100   47070907.0
2024-10-31  229.340  229.830  225.370  225.910   64370086.0

[84 rows x 5 columns]
/home/eowre/Documents/DataEngineering/AVStockTracker/SQL_data/stock_data.db
